In [ ]:
from numpwd.integrate.numeric import ExpressionMap
from numpwd.densities.h5 import read_h5
from numpwd.operators.base import Operator

from pandas import DataFrame
import pandas as pd

from sympy import sympify
import numpy as np
import os

## Read density

In [ ]:
DATA = os.path.join(os.environ["HOME"], "data", "nuc", "4he", "av18")
f = "compton-dens-4he-av18-empot-no3nf-om=0.00E+00-th=1.80E+02-nx=16-nphi=16-np12=np34=28+8-np3=36+8-nq4=nq=36+6-j12max=5-lmax=6-lsummax=14-tau4max=0-rho.h5"

In [ ]:
dens = read_h5(os.path.join(DATA, f))

In [ ]:
dens.channels

## Contact interaction

In [ ]:
expr_ss = sympify(
    "-2*(3 - 2/L**2*(p_i**2+p_o**2) + (p_i**2-p_o**2)**2/L**4)*exp(-(p_i**2+p_o**2)/L**2)"
)
expr_ss

In [ ]:
expr_ds = sympify(
    "-4*sqrt(2)/3*(2*p_i**2/L**2 + p_i**2*(p_o**2-p_i**2)/L**4)*exp(-(p_i**2+p_o**2)/L**2)"
)
expr_ds

In [ ]:
Lambda = 400  # MeV
HBARC = 197
Lambda_ifm = Lambda / HBARC
Lambda_ifm

In [ ]:
args = ("p_o", "p_i")
fc_cont = {
    "ss": ExpressionMap(expr_ss.subs({"L": Lambda_ifm}), args),
    "ds": ExpressionMap(expr_ds.subs({"L": Lambda_ifm}), args),
}

In [ ]:
fpi = 0.0924e3 / HBARC  # in i fm
LambdaB = 0.650e3 / HBARC  # i fm

nu = fpi ** 2 * LambdaB ** 3

c1s0 = -0.6 / nu
c3s0 = -1.36643 / nu
c3d1 = 2 * -0.922071 / nu

print(c1s0, c3s0, c3d1)

fc_cont_array = np.array(
    [
        c1s0 * fc_cont["ss"](dens.p, dens.p),  # 1s0, 1s0
        c3s0 * fc_cont["ss"](dens.p, dens.p),  # 3s1, 3s1
        c3d1 * fc_cont["ds"](dens.p, dens.p),  # 3d1, 3s1
        c3d1 * fc_cont["ds"](dens.p, dens.p).T,  # 3s1, 3d1
    ]
)
print(fc_cont_array.shape)

channels = DataFrame(
    [
        {"l_o": 0, "s_o": 0, "j_o": 0, "l_i": 0, "s_i": 0, "j_i": 0},  # 1s0, 1s0
        {"l_o": 0, "s_o": 1, "j_o": 1, "l_i": 0, "s_i": 1, "j_i": 1},  # 3s1, 3s1
        {"l_o": 2, "s_o": 1, "j_o": 1, "l_i": 0, "s_i": 1, "j_i": 1},  # 3d1, 3s1
        {"l_o": 0, "s_o": 1, "j_o": 1, "l_i": 2, "s_i": 1, "j_i": 1},  # 3s1, 3d1
    ]
)
channels

## Convolution

In [ ]:
isospin = {
    (0, +0, 0, +0): 1.0,
    (1, -1, 1, -1): 1.0,
    (1, +0, 1, +0): 1.0,
    (1, +1, 1, +1): 1.0,
}

op = Operator(
    matrix=fc_cont_array,
    channels=channels,
    args=tuple((arg, dens.p) for arg in args),
    isospin=isospin,
)
# op.check()

In [ ]:
p2dp = dens.p ** 2 * dens.wp
p2dp_io = p2dp.reshape(-1, 1) * p2dp
p2dp_io.shape

In [ ]:
cols = ["l_o", "s_o", "j_o", "l_i", "s_i", "j_i"]

In [ ]:
id_dens, id_op = (
    pd.merge(
        dens.channels.reset_index(),
        op.channels.reset_index(),
        how="inner",
        left_on=cols,
        right_on=cols,
        suffixes=["_dens", "_op"],
    )
    .rename(columns={"id": "id_dens", "index": "id_op"})[["id_dens", "id_op"]]
    .values.T
)
id_dens, id_op

In [ ]:
np.sum(op.matrix[id_op] * dens.matrix[id_dens] * p2dp_io) * 3 / np.pi ** 2